In [ ]:

import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 1
n_layer = 1
dropout = 0.1

In [ ]:
#set manual seed for future reproducibility
torch.manual_seed(100)

In [ ]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


--2023-01-19 14:02:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.4’

input.txt.4         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-01-19 14:02:11 (20.8 MB/s) - ‘input.txt.4’ saved [1115394/1115394]



In [52]:
path = "input.txt" 
with open(path, 'r') as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)


# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

decode(encode("hello world"))

'hello world'

In [ ]:
# create a dataset that reads input.txt and encodes it as a sequence of integers. each batch returns sequnce of size block_size
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, path, block_size, transform):
        self.transform = transform
        with open(path, 'r') as f:
            text = f.read()
        
        self.text = text
        self.block_size = block_size # here are all the unique characters that occur in this text
        self.chars = sorted(list(set(text)))
      
    def __len__(self):
        return len(self.text) - self.block_size - 1

    def __getitem__(self, idx):
        return self.transform(self.text[idx:idx+self.block_size]) , self.transform(self.text[idx+1:idx+self.block_size+1])


# define transform function that takes string list of characters and returns a tensor of integers
def transform(x):
    # convert list of characters to list of integers
    x = encode(x)
    return torch.tensor(x, dtype=torch.long)

dataset = TextDataset('input.txt', block_size , transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

print(next(iter(dataloader)))



In [ ]:
def estimate_loss(model , dataloader):
    model.eval()
    total_loss = 0
    for x,y in dataloader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            logits = model(x)
            loss = F.cross_entropy(logits.transpose(1,2), y)
            total_loss += loss.item()
    return total_loss / len(dataloader)


In [60]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, x):
        return self.emb(x)
    
    def generate(self , idx, max_new_tokens):
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [61]:
model = BigramLanguageModel().to(device)
#print number of parameters
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

4225


In [72]:
s = "my name is shubham"
s = transform(s).to(device)
s = s.unsqueeze(0)

context = torch.zeros((1,1) , dtype=torch.long).to(device)
print(decode(s.tolist()[0]))
print(decode(model.generate(s, max_new_tokens=200)[0].tolist()))


my name is shubham
my name is shubham!Z,s.ssYjL3yv JC-v qG:JVYF.tS.x:lkFzt-tjRqYg!de BITG MEHQ$SpLiWDncek&!AfB!RgszirhLTnZNne-IBMdELkrJjlS&DxgN-Cntn-cVq3g3$xp:
fLJx$ qUFL-ztN&x.3O NDFUdkfJoq,lEHoaXA$Gj3BFKOp,VU&$Fzf!X-ISnQSV
Gxg;eq!;Yc$l


In [74]:

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

epochs = max_iters // len(dataloader) + 1

for epoch in range(epochs):
    model.train()
    intervals = 0
    for i, (x,y) in enumerate(dataloader):
        intervals += 1
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = F.cross_entropy(logits.transpose(1,2), y)
        loss.backward()
        optimizer.step()
        
        #compuate loss after every eval_interval
        if intervals % eval_interval == 0:
            print(f"epoch: {epoch} , iter: {i} | train loss: {estimate_loss(model , dataloader)}")
            